In [1]:
import sys
sys.path.append('../')
from src.data_collection import get_data_from_wiki

In [2]:
enriched_topics = await get_data_from_wiki(limit=5, domain="programming")
enriched_topics

[{'id': 'Q165436',
  'title': 'assembly language',
  'wikidata_url': 'http://www.wikidata.org/entity/Q165436',
  'description': "any low-level programming language in which there is a very strong correspondence between the instructions in the language and the architecture's machine code instructions",
  'topic_type': 'programming_language',
  'properties': {'subclass of': [{'label': 'low-level programming language',
     'url': 'http://www.wikidata.org/entity/Q558937',
     'id': 'Q558937'},
    {'label': 'non-structured programming language',
     'url': 'http://www.wikidata.org/entity/Q28922854',
     'id': 'Q28922854'}],
   'instance of': [{'label': 'programming language',
     'url': 'http://www.wikidata.org/entity/Q9143',
     'id': 'Q9143'}],
   'inception': [{'label': '1949-01-01T00:00:00Z',
     'url': '1949-01-01T00:00:00Z'}]},
  'url': 'https://en.wikipedia.org/wiki/Assembly_language',
  'summary': 'In computer programming,  assembly language (alternatively assembler language

In [3]:
from src.embeddings import generate_embedding as embed_text

for topic in enriched_topics:
    topic["embedding"] = embed_text(topic["content"])

enriched_topics[0].keys()

Error generating embeddings: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


ConnectTimeout: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
from src.database.chromadb import ChromaDBClient

vector_db = ChromaDBClient()
collection = vector_db.get_or_create_collection("attempt1_programming")
# collection.delete_collection()
for topic in enriched_topics:
    collection.add(
        documents=[topic["content"]],
        metadatas=[{"title": topic["title"], "domain": topic["domain"], "timestamp": topic["timestamp"]}],
        ids=[topic["id"]],
        embeddings=[topic["embedding"]],
    )

InvalidCollectionException: Collection attempt1_programming does not exist.